In [1]:
# !pip install usd-core

In [3]:
from morphio import Morphology
from pxr import Usd, UsdGeom, Gf
import numpy as np

# Set the path to the neuron morphology file
h5_file_path = "/media/favreau/medias/morphologies/2012.07.23/dend-C030796A-P3_axon-C060110A5_-_Clone_3.h5"
morphology = Morphology(h5_file_path)

# Create a USD stage
stage = Usd.Stage.CreateNew("neuron_with_lods.usda")

# Set up metadata for the stage
stage.SetMetadata("upAxis", "Y")           # Set Y as the up axis
stage.SetMetadata("metersPerUnit", 1.0)     # Set scale to 1 unit = 1 meter

# Define the neuron hierarchy with different LODs
root = UsdGeom.Xform.Define(stage, '/Neuron')
stage.SetDefaultPrim(root.GetPrim())        # Set the defaultPrim to /Neuron

root_prim = root.GetPrim()
root_prim.SetMetadata('comment', 'This USD file contains a neuron with two levels of detail.')

# LOD 0 - Simplified representation as a single sphere positioned at the soma
lod0 = UsdGeom.Xform.Define(stage, '/Neuron/LOD0')
sphere = UsdGeom.Sphere.Define(stage, '/Neuron/LOD0/NeuronSphere')
sphere_radius = morphology.soma.diameters[0] / 2  # Assuming the first diameter represents the soma radius
sphere.GetRadiusAttr().Set(sphere_radius)

# Set the sphere position to the soma center
soma_center = morphology.soma.points[0]  # Assuming the first point is the center of the soma
sphere.AddTranslateOp().Set(Gf.Vec3f(float(soma_center[0]), float(soma_center[1]), float(soma_center[2])))

# LOD 1 - Full morphology with each section as a different curve and radii-based thickness, including the soma
lod1 = UsdGeom.Xform.Define(stage, '/Neuron/LOD1')

# Add the soma as a sphere in LOD 1
soma_sphere = UsdGeom.Sphere.Define(stage, '/Neuron/LOD1/SomaSphere')
soma_sphere.GetRadiusAttr().Set(sphere_radius)
soma_sphere.AddTranslateOp().Set(Gf.Vec3f(float(soma_center[0]), float(soma_center[1]), float(soma_center[2])))

# Add neuron sections as curves
for i, section in enumerate(morphology.root_sections):
    # For each section, create a separate curve
    curve = UsdGeom.BasisCurves.Define(stage, f'/Neuron/LOD1/NeuronCurve_{i}')
    
    # Collect all points and their corresponding radii in this section
    section_points = [Gf.Vec3f(float(p[0]), float(p[1]), float(p[2])) for p in section.points]
    radii = [d / 2 for d in section.diameters]  # Convert diameters to radii by halving
    
    # Assign points and radii-based thickness
    curve.CreatePointsAttr(section_points)
    curve.CreateCurveVertexCountsAttr([len(section_points)])  # One curve per section

    # Set the basis to 'bezier' for straight line segments between points
    curve.CreateBasisAttr().Set(UsdGeom.Tokens.bezier)
    
    # Set widths for each point, ensuring each width corresponds to a vertex in section_points
    curve.CreateWidthsAttr(radii)

# Save the USD file
stage.GetRootLayer().Save()

print("USD file with neuron LODs created successfully.")


USD file with neuron LODs created successfully.
